# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f79ccb9f910>
├── 'a' --> tensor([[-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104]])
└── 'x' --> <FastTreeValue 0x7f79ccb9fa90>
    └── 'c' --> tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                        [ 0.7096,  1.3556,  0.2609,  0.7877],
                        [ 1.4012,  0.9883, -1.0126,  0.8055]])

In [4]:
t.a

tensor([[-0.3993,  0.5711, -0.8616],
        [-1.5161, -0.8944,  0.9104]])

In [5]:
%timeit t.a

75.9 ns ± 0.219 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f79ccb9f910>
├── 'a' --> tensor([[ 2.7844,  0.0888,  2.1943],
│                   [-0.5524,  0.9798,  0.1308]])
└── 'x' --> <FastTreeValue 0x7f79ccb9fa90>
    └── 'c' --> tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                        [ 0.7096,  1.3556,  0.2609,  0.7877],
                        [ 1.4012,  0.9883, -1.0126,  0.8055]])

In [7]:
%timeit t.a = new_value

81.2 ns ± 0.387 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3993,  0.5711, -0.8616],
               [-1.5161, -0.8944,  0.9104]]),
    x: Batch(
           c: tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055]]),
       ),
)

In [10]:
b.a

tensor([[-0.3993,  0.5711, -0.8616],
        [-1.5161, -0.8944,  0.9104]])

In [11]:
%timeit b.a

71.7 ns ± 0.317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7674,  1.2302, -0.2219],
               [ 3.1176, -0.2670,  0.2041]]),
    x: Batch(
           c: tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055]]),
       ),
)

In [13]:
%timeit b.a = new_value

583 ns ± 2.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

985 ns ± 3.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f799ffd9280>
├── 'a' --> tensor([[[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]],
│           
│                   [[-0.3993,  0.5711, -0.8616],
│                    [-1.5161, -0.8944,  0.9104]]])
└── 'x' --> <FastTreeValue 0x7f7917e3bac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.6 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f79ccc0cf10>
├── 'a' --> tensor([[-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104],
│                   [-0.3993,  0.5711, -0.8616],
│                   [-1.5161, -0.8944,  0.9104]])
└── 'x' --> <FastTreeValue 0x7f79ccb64520>
    └── 'c' --> tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                        [ 0.7096,  1.3556,  0.2609,  0.7877],
                 

In [23]:
%timeit t_cat(trees)

36.4 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.4 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.4282,  0.9441,  1.3813,  0.0628],
                       [ 0.7096,  1.3556,  0.2609,  0.7877],
                       [ 1.4012,  0.9883, -1.0126,  0.8055]],
              
                      [[-2.4282,  0.9441,  1.3813,  0.0628],
                       [ 0.7096,  1.3556,  0.2609,  0.7877],
                       [ 1.4012,  0.9883, -1.0126,  0.8055]],
              
                      [[-2.4282,  0.9441,  1.3813,  0.0628],
                       [ 0.7096,  1.3556,  0.2609,  0.7877],
                       [ 1.4012,  0.9883, -1.0126,  0.8055]],
              
                      [[-2.4282,  0.9441,  1.3813,  0.0628],
                       [ 0.7096,  1.3556,  0.2609,  0.7877],
                       [ 1.4012,  0.9883, -1.0126,  0.8055]],
              
                      [[-2.4282,  0.9441,  1.3813,  0.0628],
                       [ 0.7096,  1.3556,  0.2609,  0.7877],
                       [ 1.4012,  0.9883, -1.0126,  0.8055]],

In [26]:
%timeit Batch.stack(batches)

89.5 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055],
                      [-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055],
                      [-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055],
                      [-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055],
                      [-2.4282,  0.9441,  1.3813,  0.0628],
                      [ 0.7096,  1.3556,  0.2609,  0.7877],
                      [ 1.4012,  0.9883, -1.0126,  0.8055],
                      [-2.4282,  0.9441,  1.3813,  0.0628],
                   

In [28]:
%timeit Batch.cat(batches)

164 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

366 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
